# Plot time evolution

In this notebook we make time evolution plots at a given d radius or radius range of quantities from radial profiles. 


* `ppmstarhub version: 2.0.0` or niagara hub sciconda-astrohub2 kernel
* Parent of this notebook: `ppmstar/projects/H-core-M25/analysis_paper_gas/Hcore-time-evolution-M100+.ipynb` and `Hcore-time-evolution-paperI.ipynb`
* Name of this notebook: `Rprof-time-evolution.ipynb`
* home of this notebook: https://github.com/PPMstar/PPMnotebooks/tree/master/Demos

In [ ]:
%pylab ipympl
from astropy import units
from nugridpy import utils as ut
from scipy import interpolate
import sys


scratch_dir = '/scratch/f/fherwig/fherwig/'
# scratch_dir = '/user/niagara_scratch_fherwig/'
ppmpy_dir = scratch_dir+'repos/PyPPM'
sys.path.insert(0,ppmpy_dir)


from ppmpy import ppm  
logging.getLogger("matplotlib").setLevel(logging.ERROR)


In [ ]:
# Data dictionary to hold the data for the different cases analysed in this nb
data = {}  
runs = []

Create and run below only the cells that contain runs you want to include in the current analysis. Running cells repeatedly will update the data for the corresponding runs

In [ ]:
dothis = True
# dothis = False
if dothis is True:
    nominal_heat = 0.00003355
    eos = 'g'
    
    cases = ['M109','M108','M107','M114','M115',\
              'M106','M105','M116','M110','M117','M111']
    
    data = ppm.initialize_cases(data,scratch_dir, cases, nominal_heat=nominal_heat,\
                      eos=eos,verbose = 3)

### Plot dump evolution for all runs

In [ ]:
all_cases = ppm.cases_table(data)
cases = all_cases 

In [ ]:

ifig = 113; close(ifig); figure(ifig)
last_dumps = []
for i,case in enumerate(all_cases):
    plot(data[case]['NDump'],data[case]['time(mins)']/60.,ut.linestyle(i)[0],label=case,markevery=200,\
         color=ut.linestylecb(i)[2])
    last_dumps.append(data[case]['NDump'][-1])
legend();xlabel('NDump');ylabel('t / hr')
title('marker every 200 dumps')

### Print run table

In [ ]:
ppm.cases_table(data,latex=False)

In [ ]:
# Table 1 paper I
ppm.cases_table(data,latex=True)

## RProf GUI
Use RProf GUI to do quick inspection of radial profiles.

In [ ]:
all_cases

In [ ]:
case = all_cases[0]
data[case]['rp'].rprofgui(ifig=int(case[1:3]))

### Note
The parent notebook demonstrates interpolations used in this notebook. Look there for details, removed here.

## Quantity at pressure-scale height fraction above/below N2-peak

### Time evolution at several radii given in terms of Hp above/below $N^2$ peak

This plot combines different runs, variables and radii. `var_means_dict` allows to save quantities for scaling plots.

In [ ]:
runs = ['M107', 'M114', 'M115','M111']
# runs = ['M239']

In [ ]:
# ppm.time_evol_r_Hp_vars?

In [ ]:
var_means_dict = ppm.time_evol_r_Hp_vars(data, runs, varss  = ['|Ur|'], f_hps = [1.0], key = "Demo", fname = '-Ut', logy = False,\
                        ylab="$ U_\mathrm{t} / \mathrm{[km/s]}$",  xlims=(None,None),\
                        ylims=(None,None), legends=0, NDump_range = 'time', NDump_range_vals = (150,4000),\
                        mrange_interp = (12.,14.,0.0001), sparse = 10,\
                        t_transient_hr = 500, figsizes = (8,4), ifig=1394)

In [ ]:
pwd

In [ ]:
var_means_dict

Plot below var_means. But data went into Table 2, from separate runs and data saved in `var_means_dict`.

In [ ]:
data[case].keys()

In [ ]:
# grids plot assuming there is data available for more than one grid 
### FH is in the process of debugging this ....
grids = []
grids = array(grids)
ifig=1399; close(ifig); figure(ifig,figsize=(6,3))
# plot(log10(grids),log10(var_means),'o-')
plot(log10(grids),var_means,'o-')
xlabel('$\log \mathrm{\ number\ of\ grid\ cells}$')
ylabel('$U$ in var_means')
tight_layout()